In [1]:
import pandas as pd
import numpy as np
import datetime
import os
file_paths =[r"C:\Users\Abu\data\202304-divvy-tripdata.csv", r"C:\Users\Abu\data\202305-divvy-tripdata.csv", r"C:\Users\Abu\data\202306-divvy-tripdata.csv",
            r"C:\Users\Abu\data\202307-divvy-tripdata.csv",r"C:\Users\Abu\data\202308-divvy-tripdata.csv",r"C:\Users\Abu\data\202309-divvy-tripdata.csv",
            r"C:\Users\Abu\data\202310-divvy-tripdata.csv", r"C:\Users\Abu\data\202311-divvy-tripdata.csv", r"C:\Users\Abu\data\202312-divvy-tripdata.csv",
            r"C:\Users\Abu\data\202401-divvy-tripdata.csv", r"C:\Users\Abu\data\202402-divvy-tripdata.csv", r"C:\Users\Abu\data\202403-divvy-tripdata.csv"]

# Initialize an empty list to store dataframes
dfs = []

# Iterate over each file path, read the Excel file into a dataframe, and append to the list
for file_path in file_paths:
    df = pd.read_csv(file_path)
    dfs.append(df)

# Check if any dataframes were loaded
if len(dfs) > 0:
    try:
        # Concatenate all dataframes in the list into a single dataframe
        combined_df = pd.concat(dfs, ignore_index=True)
        print("Concatenation successful. Shape of combined dataframe:", combined_df.shape)
        print(combined_df.head())  # Display the first few rows of the concatenated dataframe
    except ValueError as e:
        print("Error during concatenation:", e)
else:
    print("No Excel files found or no dataframes loaded.")
    print("File paths found:", file_paths)  # Display the list of file paths retrieved by glob.glob()

Concatenation successful. Shape of combined dataframe: (5750177, 13)
            ride_id  rideable_type           started_at             ended_at  \
0  8FE8F7D9C10E88C7  electric_bike  2023-04-02 08:37:28  2023-04-02 08:41:37   
1  34E4ED3ADF1D821B  electric_bike  2023-04-19 11:29:02  2023-04-19 11:52:12   
2  5296BF07A2F77CB5  electric_bike  2023-04-19 08:41:22  2023-04-19 08:43:22   
3  40759916B76D5D52  electric_bike  2023-04-19 13:31:30  2023-04-19 13:35:09   
4  77A96F460101AC63  electric_bike  2023-04-19 12:05:36  2023-04-19 12:10:26   

  start_station_name start_station_id end_station_name end_station_id  \
0                NaN              NaN              NaN            NaN   
1                NaN              NaN              NaN            NaN   
2                NaN              NaN              NaN            NaN   
3                NaN              NaN              NaN            NaN   
4                NaN              NaN              NaN            NaN   

   start_la

In [2]:
# Cleaning Data

In [3]:
#1. Handling missing values
# Check for missing values in each column
missing_values = combined_df.isnull().sum()
print(missing_values)

ride_id                    0
rideable_type              0
started_at                 0
ended_at                   0
start_station_name    874450
start_station_id      874450
end_station_name      929226
end_station_id        929226
start_lat                  0
start_lng                  0
end_lat                 7566
end_lng                 7566
member_casual              0
dtype: int64


In [4]:
#2: Drop rows with missing values
combined_df = combined_df.dropna(how='any', axis=0)
combined_df.isnull().sum()

ride_id               0
rideable_type         0
started_at            0
ended_at              0
start_station_name    0
start_station_id      0
end_station_name      0
end_station_id        0
start_lat             0
start_lng             0
end_lat               0
end_lng               0
member_casual         0
dtype: int64

In [5]:
# Remove duplicate rows
combined_df.duplicated().any()
#combined_df = combined_df.drop_duplicates()

False

In [6]:
# Check for any remaining missing values
print(combined_df.isnull().sum())

# Check the data types
print(combined_df.dtypes)

# Preview the cleaned dataset
print(combined_df.head())

ride_id               0
rideable_type         0
started_at            0
ended_at              0
start_station_name    0
start_station_id      0
end_station_name      0
end_station_id        0
start_lat             0
start_lng             0
end_lat               0
end_lng               0
member_casual         0
dtype: int64
ride_id                object
rideable_type          object
started_at             object
ended_at               object
start_station_name     object
start_station_id       object
end_station_name       object
end_station_id         object
start_lat             float64
start_lng             float64
end_lat               float64
end_lng               float64
member_casual          object
dtype: object
              ride_id  rideable_type           started_at  \
227  5B6500E1E58655C0   classic_bike  2023-04-10 17:34:35   
383  AA65D25D69AF771F   classic_bike  2023-04-12 12:29:46   
409  079FB2C196414482  electric_bike  2023-04-13 17:39:23   
561  599623864C871207   cla

In [7]:
# calculating ride_length

# Convert 'started_at' and 'ended_at' columns to datetime objects
combined_df['started_at'] = pd.to_datetime(combined_df['started_at'])
combined_df['ended_at'] = pd.to_datetime(combined_df['ended_at'])

# Calculate ride length (duration) in minutes
combined_df['ride_length'] = combined_df['ended_at'] - combined_df['started_at']
combined_df['ride_length'] = pd.to_numeric(combined_df["ride_length"])/6e+10
# checking for outlires or upnormal behauiour
combined_df['ride_length'].describe(include = "all")


count    4.362238e+06
mean     1.615820e+01
std      3.692482e+01
min     -5.456667e+01
25%      5.683333e+00
50%      9.916667e+00
75%      1.766667e+01
max      1.213630e+04
Name: ride_length, dtype: float64

In [8]:
# removing outliers by creating filters
outliers_trip_duration = (combined_df['ride_length'] < 1) | (combined_df['ride_length'] > 480)
outliers_trip_duration.value_counts()
combined_df.drop(combined_df.index[outliers_trip_duration], inplace = True)
combined_df['ride_length'].describe(include = "all")

count    4.274568e+06
mean     1.555436e+01
std      2.003505e+01
min      1.000000e+00
25%      5.900000e+00
50%      1.010000e+01
75%      1.786667e+01
max      4.799833e+02
Name: ride_length, dtype: float64

In [9]:
# calculating ride_distance
combined_df["ride_distance"] = np.sqrt(((combined_df["end_lat"] - combined_df["start_lat"])**2) + ((combined_df["end_lng"] - combined_df["start_lng"])**2))
combined_df["ride_distance"] = combined_df["ride_distance"] * 111.139
combined_df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,ride_length,ride_distance
227,5B6500E1E58655C0,classic_bike,2023-04-10 17:34:35,2023-04-10 18:02:36,Avenue O & 134th St,20214,Avenue O & 134th St,20214,41.651868,-87.539671,41.651868,-87.539671,member,28.016667,0.000000
383,AA65D25D69AF771F,classic_bike,2023-04-12 12:29:46,2023-04-12 12:54:00,Cottage Grove Ave & 51st St,TA1309000067,Cottage Grove Ave & 51st St,TA1309000067,41.803038,-87.606615,41.803038,-87.606615,member,24.233333,0.000000
409,079FB2C196414482,electric_bike,2023-04-13 17:39:23,2023-04-13 17:40:57,Morgan Ave & 14th Pl,TA1306000002,Morgan Ave & 14th Pl,TA1306000002,41.862430,-87.651152,41.862378,-87.651062,member,1.566667,0.011573
692,63ECC8A13D11A76A,classic_bike,2023-04-20 17:03:11,2023-04-20 17:24:58,California Ave & Division St,13256,California Ave & Milwaukee Ave,13084,41.903029,-87.697474,41.922695,-87.697153,casual,21.783333,2.185951
1093,B9F5ABDE0EC21A2D,classic_bike,2023-04-07 21:14:14,2023-04-07 21:15:14,Morgan Ave & 14th Pl,TA1306000002,Morgan Ave & 14th Pl,TA1306000002,41.862378,-87.651062,41.862378,-87.651062,member,1.000000,0.000000


In [10]:
# calculating ride_hour, day, month, year

combined_df["ride_hour"] = combined_df["started_at"].dt.hour
print(combined_df["ride_hour"].unique())

combined_df["ride_day_num"] = combined_df["started_at"].dt.dayofweek
day_dict = {0:"Sunday", 1:"Monday", 2:"Tuesday", 3:"Wednesday", 4:"Thursday", 5:"Friday", 6:"Saturday"}
combined_df["ride_day_name"]= combined_df["ride_day_num"].apply(lambda y:day_dict[y])

combined_df["weekend"] = np.where(combined_df["ride_day_num"] > 5, True, False)

combined_df['ride_month_num'] = combined_df['started_at'].dt.month
print (combined_df["ride_month_num"].unique())

month_dict = {1:"January", 2:"February", 3:"March", 4:"April", 5:"May", 6:"June", 7:"July", 8:"August", 9:"September", 10:"October", 11:"November", 12:"December"}
combined_df["ride_month_name"]= combined_df["ride_month_num"].apply(lambda y:month_dict[y])
print(combined_df["ride_month_name"].unique())

combined_df['ride_year'] = combined_df['started_at'].dt.year
print(combined_df["ride_year"].unique())

combined_df.head()

[17 12 21 11 15 16 18 14 19 13  8 10 20  4  9  3  6 22 23  0  2  1  7  5]
[ 4  5  6  7  8  9 10 11 12  1  2  3]
['April' 'May' 'June' 'July' 'August' 'September' 'October' 'November'
 'December' 'January' 'February' 'March']
[2023 2024]


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,...,member_casual,ride_length,ride_distance,ride_hour,ride_day_num,ride_day_name,weekend,ride_month_num,ride_month_name,ride_year
227,5B6500E1E58655C0,classic_bike,2023-04-10 17:34:35,2023-04-10 18:02:36,Avenue O & 134th St,20214,Avenue O & 134th St,20214,41.651868,-87.539671,...,member,28.016667,0.000000,17,0,Sunday,False,4,April,2023
383,AA65D25D69AF771F,classic_bike,2023-04-12 12:29:46,2023-04-12 12:54:00,Cottage Grove Ave & 51st St,TA1309000067,Cottage Grove Ave & 51st St,TA1309000067,41.803038,-87.606615,...,member,24.233333,0.000000,12,2,Tuesday,False,4,April,2023
409,079FB2C196414482,electric_bike,2023-04-13 17:39:23,2023-04-13 17:40:57,Morgan Ave & 14th Pl,TA1306000002,Morgan Ave & 14th Pl,TA1306000002,41.862430,-87.651152,...,member,1.566667,0.011573,17,3,Wednesday,False,4,April,2023
692,63ECC8A13D11A76A,classic_bike,2023-04-20 17:03:11,2023-04-20 17:24:58,California Ave & Division St,13256,California Ave & Milwaukee Ave,13084,41.903029,-87.697474,...,casual,21.783333,2.185951,17,3,Wednesday,False,4,April,2023
1093,B9F5ABDE0EC21A2D,classic_bike,2023-04-07 21:14:14,2023-04-07 21:15:14,Morgan Ave & 14th Pl,TA1306000002,Morgan Ave & 14th Pl,TA1306000002,41.862378,-87.651062,...,member,1.000000,0.000000,21,4,Thursday,False,4,April,2023


In [11]:
# renaming some columns
combined_df.rename(columns = {'member_casual': 'user_type', 'rideable_type' : 'bike_type'}, inplace=True)
combined_df.head()

,ride_id,bike_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,...,user_type,ride_length,ride_distance,ride_hour,ride_day_num,ride_day_name,weekend,ride_month_num,ride_month_name,ride_year
227,5B6500E1E58655C0,classic_bike,2023-04-10 17:34:35,2023-04-10 18:02:36,Avenue O & 134th St,20214,Avenue O & 134th St,20214,41.651868,-87.539671,...,member,28.016667,0.000000,17,0,Sunday,False,4,April,2023
383,AA65D25D69AF771F,classic_bike,2023-04-12 12:29:46,2023-04-12 12:54:00,Cottage Grove Ave & 51st St,TA1309000067,Cottage Grove Ave & 51st St,TA1309000067,41.803038,-87.606615,...,member,24.233333,0.000000,12,2,Tuesday,False,4,April,2023
409,079FB2C196414482,electric_bike,2023-04-13 17:39:23,2023-04-13 17:40:57,Morgan Ave & 14th Pl,TA1306000002,Morgan Ave & 14th Pl,TA1306000002,41.862430,-87.651152,...,member,1.566667,0.011573,17,3,Wednesday,False,4,April,2023
692,63ECC8A13D11A76A,classic_bike,2023-04-20 17:03:11,2023-04-20 17:24:58,California Ave & Division St,13256,California Ave & Milwaukee Ave,13084,41.903029,-87.697474,...,casual,21.783333,2.185951,17,3,Wednesday,False,4,April,2023
1093,B9F5ABDE0EC21A2D,classic_bike,2023-04-07 21:14:14,2023-04-07 21:15:14,Morgan Ave & 14th Pl,TA1306000002,Morgan Ave & 14th Pl,TA1306000002,41.862378,-87.651062,...,member,1.000000,0.000000,21,4,Thursday,False,4,April,2023


In [12]:
# exporting the dataset
combined_df.to_csv('cyclistic_dataset.csv', index=False)